# Instructions
The following code was designed to be used in conjunction with FreezeAnalysis.ipynb and FreezeAnalysis_BatchProcess.ipynb in order to set the motion threshold for detecting motion.  Videos should be loaded in which an animal is not present in the box.


### Package Requirements
The following will need to be installed in your Conda environment:

- python (3.6.5)

- jupyter

- imread

- mahotas(1.4.4)

- numpy(1.14.3)

- pandas(0.23.0)

- matplotlib(2.2.2). 

- opencv(3.4.3

The following commands can be executed in your terminal to create an environment with these packages: 

```conda config --add channels conda-forge```

```conda create -n EnvironmentName python=3.6.5 mahotas=1.4.4 pandas=0.23.0 matplotlib=2.2.2 opencv=3.4.3 jupyter imread```


---
# 1. Load Necessary Packages

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import FreezeAnalysis_Functions as fz
plt.gray()

---
# 2. Set Directory and File Information
### To be set by user:

In [ ]:
#Video information
dpath = "/Users/ZP/Desktop/Videos" # directory containing file
file = "Calibration Box 1.mpg" #filename. will take mpg and wmv files but maybe more.  only mpg1 have been extensively tested.
cal_sec = 8 #number of seconds in video to calibrate based upon
fps = 30 #frames per second

#Calibration parameters.  Need not be changed.
SIGMA = 1 #this is sigma used for gaussian smoothing of image.  Used to reduce influence of frame by frame pixel jitter. 1 works well but could increased slightly if noise is detected. 
cal_pix = 10000 #set number of pixels to be examined.  10,000 works well.


---
# 3. Load Video Information.  Display First Frame

In [ ]:
#Get full path and first frame
fpath,gray = fz.Load_First(dpath,file)

#Present frame
plt.figure(figsize=(8,8))
plt.title('first frame')
plt.imshow(gray)

---
# 4. Calibrate Video 
The following code will select cal_pix pixels (10,000, by default) at random and look at how their pixel values change across the specified length of the video.  By looking at the distribution of frame-by-frame change values for each pixel, a threshold is then set for determining what changes are likely to be attributable to the animal moving versus random fluctuation.  Currently, cutoff is is set to twice the 99.99 percentile.  

This should be performed on a video with no animal inside the box.  If multiple contexts are to be used, and freezing is to be compared across them, it is best to eventually use the same motion threshold across contexts, either taking the average of the various contexts or taking the maxiumum.  

In [ ]:
fz.Calibrate(fpath,cal_sec,cal_pix,fps,SIGMA)